# 第4单元：音频分类

现在是时候接触一些音频模型并应用您迄今为止所学到的知识了。此练习是获得课程结业证书所需的四项实践练习之一。

这是说明。marsyas/gtzan在本单元中，我们演示了如何在音乐分类数据集上微调休伯特模型。我们的示例达到了 83% 的准确率。您的任务是改进这个准确性指标。

您可以在🤗 Hub上随意选择您认为适合音频分类的任何模型，并使用完全相同的数据集marsyas/gtzan来构建您自己的分类器。

您的目标是使用您的分类器在此数据集上实现 87% 的准确率。您可以选择完全相同的模型，并使用训练超参数，或者选择完全不同的模型 - 这取决于您！

以下是一些在进行本练习时可能会有所帮助的其他资源：

Transformers 文档中的音频分类任务指南

使用 🤗 Transformer 微调音频分类

Hubert 模型文档

M-CTC-T 模型文档

音频频谱图Transformer(这个专门开一个notebooks文档)

Wav2Vec2 文档


## Transformers 文档中的音频分类任务指南

音频分类 - 就像文本一样 - 从输入数据分配一个类标签输出。唯一的区别是您拥有原始音频波形，而不是文本输入。音频分类的一些实际应用包括识别说话者意图、语言分类，甚至通过声音识别动物物种。

本指南将向您展示如何：

1. 在MInDS-14数据集上微调Wav2Vec2以对说话者意图进行分类。

2. 使用您的微调模型进行推理。

<font style=background:green>本教程中演示的任务由以下模型架构支持：
音频频谱图转换器、Data2VecAudio、Hubert、SEW、SEW-D、UniSpeech、UniSpeechSat、Wav2Vec2、Wav2Vec2-Conformer、WavLM、Whisper</font>

在开始之前，请确保已安装所有必需的库：pip install transformers datasets evaluate

我们鼓励您登录 Hugging Face 帐户，以便您可以上传模型并与社区分享。出现提示时，输入您的令牌进行登录：

In [1]:
from huggingface_hub import notebook_login

notebook_login()

### 加载 MInDS-14 数据集

首先从 🤗 数据集库加载 MInDS-14 数据集：

In [2]:
from datasets import load_dataset
minds = load_dataset("./ZZXDatasets/minds14", name="en-US", split="train")

使用train_test_split方法将数据集拆分train为较小的训练集和测试集。这将使您有机会进行试验并确保一切正常，然后再将更多时间花在完整数据集上。

In [3]:
minds = minds.train_test_split(test_size=0.2)

然后看一下数据集：

In [4]:
minds

DatasetDict({
    train: Dataset({
        features: ['path', 'audio', 'transcription', 'english_transcription', 'intent_class', 'lang_id'],
        num_rows: 450
    })
    test: Dataset({
        features: ['path', 'audio', 'transcription', 'english_transcription', 'intent_class', 'lang_id'],
        num_rows: 113
    })
})

虽然数据集包含许多有用的信息，例如lang_id和intent_class，但在本指南中english_transcription您将重点关注audio和。使用remove_columnsintent_class方法删除其他列：

In [5]:
minds = minds.remove_columns(["path", "transcription", "english_transcription", "lang_id"])

现在看一个例子：

In [6]:
minds["train"][0]

{'audio': {'path': '/home/zongxiao/.cache/huggingface/datasets/downloads/extracted/e6a84153a628f86a81aa2d7a45218ecdfb7c4336eba93c41688306bc51ad856a/en-US~PAY_BILL/602ba5a4963e11ccd901ccb8.wav',
  'array': array([ 0.00024414,  0.        ,  0.        , ..., -0.0012207 ,
         -0.0012207 ,  0.00097656]),
  'sampling_rate': 8000},
 'intent_class': 13}

有两个字段：

audioarray：必须调用以加载和重新采样音频文件的一维语音信号。
intent_class：代表说话者意图的类ID。

为了使模型更容易从标签 id 获取标签名称，请创建一个字典，将标签名称映射到整数，反之亦然：

In [7]:
labels = minds["train"].features["intent_class"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label

现在您可以将标签 id 转换为标签名称：

In [8]:
id2label[str(2)]

'app_error'

### 预处理

下一步是加载 Wav2Vec2 特征提取器来处理音频信号：

In [9]:
from transformers import AutoFeatureExtractor

feature_extractor = AutoFeatureExtractor.from_pretrained("facebook/wav2vec2-base")

/home/zongxiao/anaconda3/envs/HFAudio/lib/python3.10/site-packages/transformers/configuration_utils.py:380: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


MInDS-14 数据集的采样率为 8000khz（您可以在其数据集卡中找到此信息），这意味着您需要将数据集重新采样到 16000kHz 才能使用预训练的 Wav2Vec2 模型：

In [10]:
from datasets import Audio
minds = minds.cast_column("audio", Audio(sampling_rate=16_000))
minds["train"][0]

{'audio': {'path': '/home/zongxiao/.cache/huggingface/datasets/downloads/extracted/e6a84153a628f86a81aa2d7a45218ecdfb7c4336eba93c41688306bc51ad856a/en-US~PAY_BILL/602ba5a4963e11ccd901ccb8.wav',
  'array': array([2.09757859e-04, 1.58278723e-04, 3.78715421e-05, ...,
         1.32179086e-04, 9.02071575e-04, 6.99228728e-04]),
  'sampling_rate': 16000},
 'intent_class': 13}

现在创建一个预处理函数：

1. 调用audio列来加载音频文件，并在必要时重新采样。
2. 检查音频文件的采样率是否与模型预训练的音频数据的采样率相匹配。您可以在 Wav2Vec2模型卡中找到此信息。
3. 设置最大输入长度以批量处理较长的输入而不截断它们。

In [11]:
def preprocess_function(examples):
    audio_arrays = [x["array"] for x in examples["audio"]]
    inputs = feature_extractor(
        audio_arrays, sampling_rate=feature_extractor.sampling_rate, max_length=16000, truncation=True
    )
    return inputs

要将预处理函数应用于整个数据集，请使用 🤗 数据集映射函数。map您可以通过设置batched=True一次处理数据集的多个元素来加快速度。删除不需要的列，然后重命名intent_class为，label因为这是模型期望的名称：

In [12]:
encoded_minds = minds.map(preprocess_function, remove_columns="audio", batched=True)
encoded_minds = encoded_minds.rename_column("intent_class", "label")

Map:   0%|          | 0/450 [00:00<?, ? examples/s]

Map:   0%|          | 0/113 [00:00<?, ? examples/s]

### 评价

在训练期间包含指标通常有助于评估模型的性能。您可以使用 🤗 Evaluate库快速加载评估方法。对于此任务，加载准确性指标（请参阅 🤗 评估快速浏览以了解有关如何加载和计算指标的更多信息）：

In [13]:
import evaluate

accuracy = evaluate.load("accuracy")


然后创建一个函数，将您的预测和标签传递给计算以计算准确性：

In [14]:
import numpy as np


def compute_metrics(eval_pred):
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=eval_pred.label_ids)

您的compute_metrics函数现在已准备就绪，您将在设置训练时返回该函数。

### 训练

<span style=background:Green>T如果您不熟悉使用Trainer微调模型，请查看此处的基本教程！</span>

您现在就可以开始训练您的模型了！使用AutoModelForAudioClassification加载 Wav2Vec2以及预期标签的数量和标签映射：

In [15]:
from transformers import AutoModelForAudioClassification, TrainingArguments, Trainer

num_labels = len(id2label)
model = AutoModelForAudioClassification.from_pretrained(
    "facebook/wav2vec2-base", num_labels=num_labels, label2id=label2id, id2label=id2label
)

/home/zongxiao/anaconda3/envs/HFAudio/lib/python3.10/site-packages/transformers/configuration_utils.py:380: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['projector.weight', 'projector.bias', 'classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


此时，只剩下三步：

1. 在TrainingArguments中定义训练超参数。唯一必需的参数是output_dir指定保存模型的位置。您可以通过设置将此模型推送到 Hub push_to_hub=True（您需要登录 Hugging Face 才能上传模型）。在每个 epoch 结束时，Trainer将评估准确性并保存训练检查点。

2. 将训练参数以及模型、数据集、分词器、数据整理器和函数传递给
Trainercompute_metrics。

3. 调用train()来微调您的模型。

In [16]:
training_args = TrainingArguments(
    output_dir="my_awesome_mind_model",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=32,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=32,
    num_train_epochs=10,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_minds["train"],
    eval_dataset=encoded_minds["test"],
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
0,No log,2.644150,0.088496
1,No log,2.650759,0.053097
2,2.636000,2.652251,0.017699
4,2.636000,2.646824,0.097345
5,2.620500,2.650995,0.079646
6,2.620500,2.648434,0.079646
8,2.615400,2.648645,0.088496


TrainOutput(global_step=30, training_loss=2.6239901860555013, metrics={'train_runtime': 120.81, 'train_samples_per_second': 37.249, 'train_steps_per_second': 0.248, 'total_flos': 3.26841433344e+16, 'train_loss': 2.6239901860555013, 'epoch': 8.0})

训练完成后，使用push_to_hub()方法将您的模型共享到 Hub ，以便每个人都可以使用您的模型：

In [17]:
trainer.push_to_hub()

pytorch_model.bin:   0%|          | 0.00/378M [00:00<?, ?B/s]

'https://huggingface.co/zongxiao/my_awesome_mind_model/tree/main/'

### 推理

太好了，现在您已经微调了模型，您可以使用它进行推理！

加载您想要运行推理的音频文件。如果需要，请记住重新采样音频文件的采样率以匹配模型的采样率！

In [45]:
from datasets import load_dataset, Audio

dataset = load_dataset("./ZZXDatasets/minds14", name="en-US", split="train")
dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))
sampling_rate = dataset.features["audio"].sampling_rate
audio_file = dataset[0]["audio"]["path"]

In [47]:
from huggingface_hub import notebook_login

notebook_login()

In [50]:
from transformers import pipeline

classifier = pipeline("audio-classification", model="zongxiao/my_awesome_mind_model")
classifier(audio_file)

[{'score': 0.08231057971715927, 'label': 'card_issues'},
 {'score': 0.07625565677881241, 'label': 'freeze'},
 {'score': 0.07588177174329758, 'label': 'cash_deposit'},
 {'score': 0.07493145018815994, 'label': 'high_value_payment'},
 {'score': 0.07455172389745712, 'label': 'pay_bill'}]

pipeline如果您愿意，您还可以手动复制结果：

#### Pytorch

加载特征提取器来预处理音频文件并返回inputPyTorch 张量：

In [52]:
from transformers import AutoFeatureExtractor

feature_extractor = AutoFeatureExtractor.from_pretrained("zongxiao/my_awesome_mind_model")
inputs = feature_extractor(dataset[0]["audio"]["array"], sampling_rate=sampling_rate, return_tensors="pt")

将您的输入传递给模型并返回 logits：

In [54]:
from transformers import AutoModelForAudioClassification

model = AutoModelForAudioClassification.from_pretrained("zongxiao/my_awesome_mind_model")
with torch.no_grad():
    logits = model(**inputs).logits

获取概率最高的类，并使用模型的id2label映射将其转换为标签：

In [55]:
import torch

predicted_class_ids = torch.argmax(logits).item()
predicted_label = model.config.id2label[predicted_class_ids]
predicted_label

'card_issues'

## 使用 🤗 Transformer 微调音频分类

本笔记本展示了如何微调用于自动语音识别的多语言预训练语音模型。

该笔记本构建为在SUPERB 数据集的关键字识别子集上运行，具有来自模型中心的任何语音模型检查点，只要该模型具有带有序列分类头的版本（例如Wav2Vec2ForSequenceClassification）。

根据您使用的型号和 GPU，您可能需要调整批量大小以避免内存不足错误。设置这两个参数，然后笔记本的其余部分应该可以顺利运行：

In [18]:
model_checkpoint = "facebook/wav2vec2-base"
batch_size = 32

In [19]:
from huggingface_hub import notebook_login

notebook_login()

我们还快速上传一些遥测数据 - 这告诉我们正在使用哪些示例和软件版本，以便我们知道在哪里优先考虑我们的维护工作。我们不收集（或关心）任何个人身份信息，但如果您不想被统计在内，请随时跳过此步骤或完全删除此单元格。

In [20]:
from transformers.utils import send_example_telemetry

send_example_telemetry("audio_classification_notebook", framework="pytorch")

### 在音频分类任务上微调模型

在本笔记本中，我们将了解如何将🤗 Transformers声学模型之一微调为SUPERB Benchmark的关键字识别任务

关键字识别 (KS) 通过将话语分类为一组预定义的单词来检测预先注册的关键字。SUPERB 使用广泛使用的语音命令数据集 v1.0 来完成该任务。该数据集由十类关键字、一类用于沉默的类和一个包含误报的未知类组成。

#### 加载数据集

我们将使用🤗数据集库下载数据并获取我们需要用于评估的准确性指标。这可以通过函数load_dataset和轻松完成load_metric。

In [21]:
from datasets import load_dataset, load_metric

In [22]:
dataset = load_dataset("superb", "ks")
metric = load_metric("accuracy")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/51094 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/6798 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3081 [00:00<?, ? examples/s]

/tmp/ipykernel_4714/35737003.py:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")


该dataset对象本身是一个DatasetDict，其中包含一个用于训练、验证和测试集的密钥。

In [23]:
dataset

DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'label'],
        num_rows: 51094
    })
    validation: Dataset({
        features: ['file', 'audio', 'label'],
        num_rows: 6798
    })
    test: Dataset({
        features: ['file', 'audio', 'label'],
        num_rows: 3081
    })
})

要访问实际元素，您需要先选择一个拆分，然后给出一个索引：

In [24]:
dataset["test"][1000]

{'file': '/home/zongxiao/.cache/huggingface/datasets/downloads/extracted/547529e0aa90b2ae823bce8003e7322f0883ef626dda4e40ddfb90b3f3aec7f8/go/e41a903b_nohash_4.wav',
 'audio': {'path': '/home/zongxiao/.cache/huggingface/datasets/downloads/extracted/547529e0aa90b2ae823bce8003e7322f0883ef626dda4e40ddfb90b3f3aec7f8/go/e41a903b_nohash_4.wav',
  'array': array([-1.22070312e-04,  3.05175781e-05,  1.83105469e-04, ...,
         -4.88281250e-04, -5.49316406e-04, -3.35693359e-04]),
  'sampling_rate': 16000},
 'label': 9}

如您所见，该label字段不是实际的字符串标签。默认情况下，ClassLabel为了方便起见，字段被编码为整数：

In [25]:
dataset["train"].features["label"]

ClassLabel(names=['yes', 'no', 'up', 'down', 'left', 'right', 'on', 'off', 'stop', 'go', '_silence_', '_unknown_'], id=None)

让我们创建一个id2label字典来将它们解码回字符串并看看它们是什么。label2id当我们稍后加载模型时，逆函数也很有用。

In [26]:
labels = dataset["train"].features["label"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label

id2label["9"]

'go'

Wav2Vec2期望输入采用 16 kHz 的一维数组格式。这意味着必须加载音频文件并重新采样。

值得庆幸的是，datasets在调用 column 时会自动执行此操作audio。让我们尝试一下吧。

In [27]:
dataset["test"][1000]["audio"]

{'path': '/home/zongxiao/.cache/huggingface/datasets/downloads/extracted/547529e0aa90b2ae823bce8003e7322f0883ef626dda4e40ddfb90b3f3aec7f8/go/e41a903b_nohash_4.wav',
 'array': array([-1.22070312e-04,  3.05175781e-05,  1.83105469e-04, ...,
        -4.88281250e-04, -5.49316406e-04, -3.35693359e-04]),
 'sampling_rate': 16000}

我们可以看到音频文件已经自动加载了。这要归功于中引入的新"Audio"功能datasets == 1.13.3，该功能可以在呼叫时即时加载和重新采样音频文件。

采样率设置为 16kHz，这是Wav2Vec2输入的预期值。

为了了解命令听起来是什么样的，以下代码片段将渲染从数据集中随机选取的一些音频示例。

注意：您可以运行以下单元几次来收听不同的音频样本。

In [28]:
import random
from IPython.display import Audio, display

for _ in range(5):
    rand_idx = random.randint(0, len(dataset["train"])-1)
    example = dataset["train"][rand_idx]
    audio = example["audio"]

    print(f'Label: {id2label[str(example["label"])]}')
    print(f'Shape: {audio["array"].shape}, sampling rate: {audio["sampling_rate"]}')
    display(Audio(audio["array"], rate=audio["sampling_rate"]))
    print()

Label: _unknown_
Shape: (11606,), sampling rate: 16000



Label: down
Shape: (16000,), sampling rate: 16000



Label: _unknown_
Shape: (16000,), sampling rate: 16000



Label: yes
Shape: (16000,), sampling rate: 16000



Label: left
Shape: (16000,), sampling rate: 16000


如果您运行该单元几次，您会发现尽管长度略有变化，但大多数样本的长度约为 1 秒 ( duration = audio_length / sampling_rate)。因此我们可以安全地将样本截断并填充到16000.

#### 预处理数据

在我们将这些音频剪辑输入到我们的模型之前，我们需要对它们进行预处理。这是由🤗 Transformers 完成的FeatureExtractor，它将规范化输入并将其设置为模型期望的格式，并生成模型所需的其他输入。

为了完成所有这些，我们使用该AutoFeatureExtractor.from_pretrained方法实例化我们的特征提取器，这将确保我们获得与我们想要使用的模型架构相对应的预处理器。

In [30]:
from transformers import AutoFeatureExtractor

feature_extractor = AutoFeatureExtractor.from_pretrained(model_checkpoint)
feature_extractor

/home/zongxiao/anaconda3/envs/HFAudio/lib/python3.10/site-packages/transformers/configuration_utils.py:380: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


Wav2Vec2FeatureExtractor {
  "do_normalize": true,
  "feature_extractor_type": "Wav2Vec2FeatureExtractor",
  "feature_size": 1,
  "padding_side": "right",
  "padding_value": 0.0,
  "return_attention_mask": false,
  "sampling_rate": 16000
}

正如我们之前注意到的，样本大约有 1 秒长，所以我们在这里设置：

In [31]:
max_duration = 1.0  # seconds

然后我们可以编写对样本进行预处理的函数。我们只需将feature_extractor参数truncation=True以及最大样本长度提供给它们。这将确保像类中的输入一样的非常长的输入_silence_可以安全地进行批处理。

In [32]:
def preprocess_function(examples):
    audio_arrays = [x["array"] for x in examples["audio"]]
    inputs = feature_extractor(
        audio_arrays, 
        sampling_rate=feature_extractor.sampling_rate, 
        max_length=int(feature_extractor.sampling_rate * max_duration), 
        truncation=True, 
    )
    return inputs

特征提取器将为每个示例返回一个 numpy 数组列表：

In [33]:
preprocess_function(dataset['train'][:5])

{'input_values': [array([-9.1631009e-05, -9.1631009e-05, -9.1631009e-05, ...,
       -4.6719767e-02, -8.0353022e-01, -1.3182331e+00], dtype=float32), array([0.01049979, 0.01049979, 0.01049979, ..., 0.6454253 , 0.43378347,
       0.25741526], dtype=float32), array([ 9.0340059e-04,  9.0340059e-04,  9.0340059e-04, ...,
       -1.7281245e-01,  2.2313449e-01,  1.9931581e+00], dtype=float32), array([ 1.5586768 ,  0.3870289 ,  0.74101615, ..., -0.8897349 ,
       -0.7703889 , -0.09471782], dtype=float32), array([-0.01518929, -0.01518929, -0.01518929, ..., -0.84138   ,
        0.22227868, -0.02409434], dtype=float32)]}

要将此函数应用于数据集中的所有话语，我们只需使用之前创建的对象map的方法即可。dataset这会将函数应用于 中所有分割的所有元素dataset，因此我们的训练、验证和测试数据将在一个命令中进行预处理。

In [34]:
encoded_dataset = dataset.map(preprocess_function, remove_columns=["audio", "file"], batched=True)
encoded_dataset

Map:   0%|          | 0/51094 [00:00<?, ? examples/s]

Map:   0%|          | 0/6798 [00:00<?, ? examples/s]

Map:   0%|          | 0/3081 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'input_values'],
        num_rows: 51094
    })
    validation: Dataset({
        features: ['label', 'input_values'],
        num_rows: 6798
    })
    test: Dataset({
        features: ['label', 'input_values'],
        num_rows: 3081
    })
})

更好的是，结果会由 🤗 数据集库自动缓存，以避免下次运行笔记本时在此步骤上花费时间。🤗 数据集库通常足够智能，可以检测您传递给映射的函数何时发生更改（因此需要不使用缓存数据）。🤗 Datasets 在使用缓存文件时会警告您，您可以传入load_from_cache_file=False调用以map不使用缓存文件并强制再次应用预处理。

#### 训练模型

现在我们的数据已准备就绪，我们可以下载预训练的模型并对其进行微调。对于分类，我们使用AutoModelForAudioClassification类。与特征提取器一样，该from_pretrained方法将为我们下载并缓存模型。由于标签 ID 和标签数量取决于数据集，因此我们在此处传递num_labels、label2id和：id2labelmodel_checkpoint

In [35]:
from transformers import AutoModelForAudioClassification, TrainingArguments, Trainer

num_labels = len(id2label)
model = AutoModelForAudioClassification.from_pretrained(
    model_checkpoint, 
    num_labels=num_labels,
    label2id=label2id,
    id2label=id2label,
)

Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['projector.weight', 'projector.bias', 'classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


该警告告诉我们，我们正在丢弃一些权重（quantizer和project_q层）并随机初始化其他一些权重（projector和classifier层）。在这种情况下，这是预期的，因为我们正在删除用于在无监督矢量量化目标上预训练模型的头，并将其替换为我们没有预训练权重的新头，因此库警告我们应该罚款-在使用模型进行推理之前对其进行调整，这正是我们要做的。

要实例化Trainer，我们需要定义训练配置和评估指标。最重要的是TrainingArguments，它是一个包含自定义训练的所有属性的类。它需要一个文件夹名称，该名称将用于保存模型的检查点，所有其他参数都是可选的：

In [36]:
model_name = model_checkpoint.split("/")[-1]

args = TrainingArguments(
    f"{model_name}-finetuned-ks",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=True,
)

在这里，我们设置在每个 epoch 结束时进行评估，调整学习率，使用batch_size笔记本顶部的定义并自定义训练的 epoch 数以及权重衰减。由于最佳模型可能不是训练结束时的模型，因此我们要求加载训练结束时Trainer保存的最佳模型（根据）。metric_name

最后一个参数push_to_hub允许 Trainer 在训练期间定期将模型推送到Hub。如果您没有按照笔记本顶部的安装步骤进行操作，请将其删除。如果您想使用与存储库名称不同的名称在本地保存模型，或者如果您想将模型推送到组织而不是您的名称空间下，请使用参数来设置存储库名称（hub_model_id需要是全名，包括您的命名空间：例如"anton-l/wav2vec2-finetuned-ks"或"huggingface/anton-l/wav2vec2-finetuned-ks")。

接下来，我们需要定义一个函数来计算预测的指标，该函数将仅使用metric我们之前加载的数据。我们唯一要做的预处理就是获取预测 logits 的 argmax：

In [37]:
import numpy as np

def compute_metrics(eval_pred):
    """Computes accuracy on a batch of predictions"""
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return metric.compute(predictions=predictions, references=eval_pred.label_ids)

然后我们只需将所有这些以及我们的数据集传递到Trainer：

In [38]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics
)

您可能想知道为什么当我们已经预处理了数据时我们还要将 传递为feature_extractor分词器。这是因为我们将在最后一次使用它，通过应用填充来使我们收集的所有样本具有相同的长度，这需要了解模型对填充的偏好（向左还是向右？使用哪个标记？）。有feature_extractor一个 pad 方法可以为我们完成所有这些工作，并且Trainer将使用它。您可以通过定义和传递您自己的部分来自定义此部分，data_collator该部分将接收像上面看到的字典一样的样本，并且需要返回张量字典。

现在我们可以通过调用以下train方法来微调我们的模型：

In [39]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
0,0.692400,0.581311,0.888497
1,0.256800,0.179234,0.977640
2,0.204800,0.112561,0.980435
4,0.163700,0.092447,0.981612


TrainOutput(global_step=1995, training_loss=0.46572097808794866, metrics={'train_runtime': 19595.5511, 'train_samples_per_second': 13.037, 'train_steps_per_second': 0.102, 'total_flos': 2.3180194781952e+18, 'train_loss': 0.46572097808794866, 'epoch': 5.0})

我们可以使用该方法检查evaluate我们Trainer是否正确重新加载了最佳模型（如果不是最后一个）：

In [41]:
trainer.evaluate()

{'eval_loss': 0.09421850740909576,
 'eval_accuracy': 0.9824948514268903,
 'eval_runtime': 72.4672,
 'eval_samples_per_second': 93.808,
 'eval_steps_per_second': 2.939,
 'epoch': 5.0}

您现在可以将训练结果上传到 Hub，只需执行以下指令：

In [42]:
trainer.push_to_hub()

pytorch_model.bin:   0%|          | 0.00/378M [00:00<?, ?B/s]

'https://huggingface.co/zongxiao/wav2vec2-base-finetuned-ks/tree/main/'

您现在可以与所有朋友、家人、最喜欢的宠物共享此模型：他们都可以使用标识符加载它，"your-username/the-name-you-picked"例如：

In [44]:
from transformers import AutoModelForAudioClassification, AutoFeatureExtractor

feature_extractor = AutoFeatureExtractor.from_pretrained("zongxiao/wav2vec2-base-finetuned-ks")
model = AutoModelForAudioClassification.from_pretrained("zongxiao/wav2vec2-base-finetuned-ks")

## Hubert 模型文档

### 概述

Hubert 是由 Wei-Ning Hsu、Benjamin Bolte、Yao-Hung Hubert Tsai、Kushal Lakhotia、Ruslan Salakhutdinov、Abdelrahman Mohamed在HuBERT：通过隐藏单元的掩蔽预测进行自我监督语音表征学习中提出的。

论文摘要如下：

用于语音表示学习的自监督方法面临三个独特问题的挑战：（1）每个输入话语中有多个声音单元，（2）在预训练阶段没有输入声音单元的词典，以及（3）声音单元的长度可变，没有明确的分段。为了解决这三个问题，我们提出了用于自监督语音表示学习的隐藏单元 BERT (HuBERT) 方法，该方法利用离线聚类步骤为类似 BERT 的预测损失提供对齐的目标标签。我们方法的一个关键要素是仅在屏蔽区域应用预测损失，这迫使模型在连续输入上学习组合的声学和语言模型。HuBERT 主要依赖于无监督聚类步骤的一致性，而不是分配的聚类标签的内在质量。从 100 个聚类的简单 k 均值教师开始，并使用两次聚类迭代，HuBERT 模型匹配或改进了 Librispeech (960h) 和 Libri-light (60,000) 上最先进的 wav2vec 2.0 性能h) 具有 10min、1h、10h、100h 和 960h 微调子集的基准。使用 1B 参数模型，HuBERT 在更具挑战性的开发其他和测试其他评估子集上显示相对 WER 降低高达 19% 和 13%。在 Librispeech (960h) 和 Libri-light (60,000h) 基准测试中，具有 10min、1h、10h、100h 和 960h 微调子集的 0 性能。使用 1B 参数模型，HuBERT 在更具挑战性的开发其他和测试其他评估子集上显示相对 WER 降低高达 19% 和 13%。在 Librispeech (960h) 和 Libri-light (60,000h) 基准测试中，具有 10min、1h、10h、100h 和 960h 微调子集的 0 性能。使用 1B 参数模型，HuBERT 在更具挑战性的开发其他和测试其他评估子集上显示相对 WER 降低高达 19% 和 13%。

Tips:

Hubert 是一个语音模型，它接受与语音信号的原始波形相对应的浮点数组。

Hubert 模型使用联结时序分类 (CTC) 进行微调，因此模型输出必须使用Wav2Vec2CTCTokenizer进行解码。



### HubertConfig


#### class transformers.HubertConfig

<
source
>
( vocab_size = 32hidden_size = 768num_hidden_layers = 12num_attention_heads = 12intermediate_size = 3072hidden_act = 'gelu'hidden_dropout = 0.1activation_dropout = 0.1attention_dropout = 0.1feat_proj_layer_norm = Truefeat_proj_dropout = 0.0final_dropout = 0.1layerdrop = 0.1initializer_range = 0.02layer_norm_eps = 1e-05feat_extract_norm = 'group'feat_extract_activation = 'gelu'conv_dim = (512, 512, 512, 512, 512, 512, 512)conv_stride = (5, 2, 2, 2, 2, 2, 2)conv_kernel = (10, 3, 3, 3, 3, 2, 2)conv_bias = Falsenum_conv_pos_embeddings = 128num_conv_pos_embedding_groups = 16do_stable_layer_norm = Falseapply_spec_augment = Truemask_time_prob = 0.05mask_time_length = 10mask_time_min_masks = 2mask_feature_prob = 0.0mask_feature_length = 10mask_feature_min_masks = 0ctc_loss_reduction = 'sum'ctc_zero_infinity = Falseuse_weighted_layer_sum = Falseclassifier_proj_size = 256pad_token_id = 0bos_token_id = 1eos_token_id = 2**kwargs )

#### Parameters

vocab_size (int, optional, defaults to 32) — Vocabulary size of the Hubert model. Defines the number of different tokens that can be represented by the inputs_ids passed when calling HubertModel. Vocabulary size of the model. Defines the different tokens that can be represented by the inputs_ids passed to the forward method of HubertModel.— Hubert 模型的词汇大小。定义调用HubertModelinputs_ids时传递的可以表示的不同令牌的数量 。模型的词汇量。定义可以由传递给HubertModel的forward方法的inputs_ids表示的不同标记。

hidden_size (int, optional, defaults to 768) — Dimensionality of the encoder layers and the pooler layer.— 编码器层和池化层的维度

num_hidden_layers (int, optional, defaults to 12) — Number of hidden layers in the Transformer encoder.— Transformer 编码器中隐藏层的数量。

num_attention_heads (int, optional, defaults to 12) — Number of attention heads for each attention layer in the Transformer encoder.— Transformer 编码器中每个注意力层的注意力头数量。

intermediate_size (int, optional, defaults to 3072) — Dimensionality of the “intermediate” (i.e., feed-forward) layer in the Transformer encoder.— Transformer 编码器中“中间”（即前馈）层的维度。

hidden_act (str or function, optional, defaults to "gelu") — The non-linear activation function (function or string) in the encoder and pooler. If string, "gelu", "relu", "selu" and "gelu_new" are supported.— 编码器和池化器中的非线性激活函数（函数或字符串）。如果支持字符串、"gelu"、 "relu"、"selu"和。"gelu_new"

hidden_dropout(float, optional, defaults to 0.1) — The dropout probability for all fully connected layers in the embeddings, encoder, and pooler. — 嵌入、编码器和池化器中所有全连接层的丢失概率。

attention_dropout(float, optional, defaults to 0.1) — The dropout ratio for the attention probabilities.— 注意力概率的丢失率。

final_dropout (float, optional, defaults to 0.1) — The dropout probabilitiy for the final projection layer of Wav2Vec2ForCTC.— Wav2Vec2ForCTC最终投影层的 dropout 概率。

layerdrop (float, optional, defaults to 0.1) — The LayerDrop probability. See the [LayerDrop paper](see https://arxiv.org/abs/1909.11556) for more details.— LayerDrop 概率。更多细节请参见[LayerDrop论文]（参见https://arxiv.org/abs/1909.11556 ）。

initializer_range (float, optional, defaults to 0.02) — The standard deviation of the truncated_normal_initializer for initializing all weight matrices.initializer_range ( float，可选，默认为 0.02) — 用于初始化所有权重矩阵的 truncated_normal_initializer 的标准差。

layer_norm_eps (float, optional, defaults to 1e-12) — The epsilon used by the layer normalization layers. — 层归一化层使用的 epsilon

feat_extract_norm (str, optional, defaults to "group") — The norm to be applied to 1D convolutional layers in feature encoder. One of "group" for group normalization of only the first 1D convolutional layer or "layer" for layer normalization of all 1D convolutional layers.— 应用于特征编码器中 1D 卷积层的范数。一个"group"用于仅第一个一维卷积层的组归一化，或"layer"用于所有一维卷积层的层归一化。

feat_proj_dropout (float, optional, defaults to 0.0) — The dropout probability for output of the feature encoder.— 特征编码器输出的丢失概率。

feat_proj_layer_norm (bool, optional, defaults to True) — Whether to apply LayerNorm to the output of the feature encoder.— 是否将 LayerNorm 应用于特征编码器的输出。

feat_extract_activation (str, optional, defaults to “gelu”) -- The non-linear activation function (function or string) in the 1D convolutional layers of the feature extractor. If string, “gelu”, “relu”, “selu”and“gelu_new”` are supported.

conv_dim (Tuple[int], optional, defaults to (512, 512, 512, 512, 512, 512, 512)) — A tuple of integers defining the number of input and output channels of each 1D convolutional layer in the feature encoder. The length of conv_dim defines the number of 1D convolutional layers.— 一个整数元组，定义特征编码器中每个 1D 卷积层的输入和输出通道的数量。conv_dim的长度定义一维卷积层的数量。

conv_stride (Tuple[int], optional, defaults to (5, 2, 2, 2, 2, 2, 2)) — A tuple of integers defining the stride of each 1D convolutional layer in the feature encoder. The length of conv_stride defines the number of convolutional layers and has to match the length of conv_dim.— 定义特征编码器中每个 1D 卷积层的步幅的整数元组。conv_stride的长度定义了卷积层的数量，并且必须与conv_dim的长度匹配。

conv_kernel (Tuple[int], optional, defaults to (10, 3, 3, 3, 3, 3, 3)) — A tuple of integers defining the kernel size of each 1D convolutional layer in the feature encoder. The length of conv_kernel defines the number of convolutional layers and has to match the length of conv_dim.— 定义特征编码器中每个 1D 卷积层的内核大小的整数元组。conv_kernel的长度定义了卷积层的数量，并且必须与 conv_dim的长度匹配。

conv_bias (bool, optional, defaults to False) — Whether the 1D convolutional layers have a bias.— 一维卷积层是否有偏差。

num_conv_pos_embeddings (int, optional, defaults to 128) — Number of convolutional positional embeddings. Defines the kernel size of 1D convolutional positional embeddings layer.— 卷积位置嵌入的数量。定义一维卷积位置嵌入层的内核大小。

num_conv_pos_embedding_groups (int, optional, defaults to 16) — Number of groups of 1D convolutional positional embeddings layer.— 1D 卷积位置嵌入层的组数。

do_stable_layer_norm (bool, optional, defaults to False) — Whether do apply stable layer norm architecture of the Transformer encoder. do_stable_layer_norm is True corresponds to applying layer norm before the attention layer, whereas do_stable_layer_norm is False corresponds to applying layer norm after the attention layer.— 是否应用Transformer 编码器的稳定层规范架构。do_stable_layer_norm is True对应于在注意层之前应用层范数，而do_stable_layer_norm is False对应于在注意层之后应用层范数。

apply_spec_augment (bool, optional, defaults to True) — Whether to apply SpecAugment data augmentation to the outputs of the feature encoder. For reference see SpecAugment: A Simple Data Augmentation Method for Automatic Speech Recognition.— 是否将SpecAugment数据增强应用于特征编码器的输出。有关参考，请参阅 SpecAugment：自动语音识别的简单数据增强方法。

mask_time_prob (float, optional, defaults to 0.05) — Percentage (between 0 and 1) of all feature vectors along the time axis which will be masked. The masking procecure generates ”mask_time_problen(time_axis)/mask_time_length” independent masks over the axis. If reasoning from the propability of each feature vector to be chosen as the start of the vector span to be masked, mask_time_prob should be `prob_vector_startmask_time_length. Note that overlap may decrease the actual percentage of masked vectors. This is only relevant if apply_spec_augment is True`.

mask_time_length (int, optional, defaults to 10) — Length of vector span along the time axis.— 沿时间轴的矢量跨度长度。

mask_time_min_masks (int, optional, defaults to 2), — The minimum number of masks of length mask_feature_length generated along the time axis, each time step, irrespectively of mask_feature_prob. Only relevant if ”mask_time_prob*len(time_axis)/mask_time_length < mask_time_min_masks”—每个时间步沿时间轴生成的长度掩码的最小数量，与 无关mask_feature_prob。仅当“mask_time_prob*len(time_axis)/mask_time_length < mask_time_min_masks”时才相关

mask_feature_prob (float, optional, defaults to 0.0) — Percentage (between 0 and 1) of all feature vectors along the feature axis which will be masked. The masking procecure generates ”mask_feature_problen(feature_axis)/mask_time_length” independent masks over the axis. If reasoning from the propability of each feature vector to be chosen as the start of the vector span to be masked, mask_feature_prob should be `prob_vector_startmask_feature_length. Note that overlap may decrease the actual percentage of masked vectors. This is only relevant if apply_spec_augment is True`.

mask_feature_length (int, optional, defaults to 10) — Length of vector span along the feature axis.— 沿特征轴的向量跨度的长度。

mask_feature_min_masks (int, optional, defaults to 0), — The minimum number of masks of length mask_feature_length generated along the feature axis, each time step, irrespectively of mask_feature_prob. Only relevant if ”mask_feature_prob*len(feature_axis)/mask_feature_length < mask_feature_min_masks”— 每个时间步沿特征轴生成的长度掩码的最小数量，mask_feature_length与 无关mask_feature_prob。仅当“mask_feature_prob*len(feature_axis)/mask_feature_length < mask_feature_min_masks”时才相关

ctc_loss_reduction (str, optional, defaults to "sum") — Specifies the reduction to apply to the output of torch.nn.CTCLoss. Only relevant when training an instance of HubertForCTC.— 指定应用于 输出的减少量torch.nn.CTCLoss。仅在训练HubertForCTC实例时相关。

ctc_zero_infinity (bool, optional, defaults to False) — Whether to zero infinite losses and the associated gradients of torch.nn.CTCLoss. Infinite losses mainly occur when the inputs are too short to be aligned to the targets. Only relevant when training an instance of HubertForCTC.
— 是否将 的无限损失和相关梯度归零torch.nn.CTCLoss。当输入太短而无法与目标保持一致时，就会出现无限损失。仅在训练HubertForCTC实例时相关。

use_weighted_layer_sum (bool, optional, defaults to False) — Whether to use a weighted average of layer outputs with learned weights. Only relevant when using an instance of HubertForSequenceClassification.— 是否使用具有学习权重的层输出的加权平均值。仅在使用HubertForSequenceClassification实例时相关。

classifier_proj_size (int, optional, defaults to 256) — Dimensionality of the projection before token mean-pooling for classification.— 用于分类的标记均值池之前投影的维数。

这是存储HubertModel配置的配置类。它用于根据指定的参数实例化 Hubert 模型，定义模型架构。使用默认值实例化配置将产生与 Hubert facebook/hubert-base-ls960架构类似的配置。

配置对象继承自PretrainedConfig，可用于控制模型输出。阅读PretrainedConfig的文档以获取更多信息。

#### Example:

In [60]:
from transformers import HubertModel, HubertConfig

# Initializing a Hubert facebook/hubert-base-ls960 style configuration
configuration = HubertConfig()
print(configuration) 

# Initializing a model from the facebook/hubert-base-ls960 style configuration
model = HubertModel(configuration)

# Accessing the model configuration
configuration = model.config
print(configuration) 

HubertConfig {
  "activation_dropout": 0.1,
  "apply_spec_augment": true,
  "attention_dropout": 0.1,
  "bos_token_id": 1,
  "classifier_proj_size": 256,
  "conv_bias": false,
  "conv_dim": [
    512,
    512,
    512,
    512,
    512,
    512,
    512
  ],
  "conv_kernel": [
    10,
    3,
    3,
    3,
    3,
    2,
    2
  ],
  "conv_stride": [
    5,
    2,
    2,
    2,
    2,
    2,
    2
  ],
  "ctc_loss_reduction": "sum",
  "ctc_zero_infinity": false,
  "do_stable_layer_norm": false,
  "eos_token_id": 2,
  "feat_extract_activation": "gelu",
  "feat_extract_norm": "group",
  "feat_proj_dropout": 0.0,
  "feat_proj_layer_norm": true,
  "final_dropout": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "layerdrop": 0.1,
  "mask_feature_length": 10,
  "mask_feature_min_masks": 0,
  "mask_feature_prob": 0.0,
  "mask_time_length": 10,
  "mask_time_min_masks": 2,
  "mask_ti

### HubertModel

#### class transformers.HubertModel

参数

config ( HuberConfig ) — 包含模型所有参数的模型配置类。使用配置文件初始化不会加载与模型关联的权重，仅加载配置。查看from_pretrained()方法来加载模型权重。

config ( HuberConfig ) — 包含模型所有参数的模型配置类。使用配置文件初始化不会加载与模型关联的权重，仅加载配置。查看from_pretrained()方法来加载模型权重。

裸露的Hubert模型transformer输出原始隐藏状态，顶部没有任何特定的头。Hubert 是由 Wei-Ning Hsu、Benjamin Bolte、Yao-Hung Hubert Tsai、Kushal Lakhotia、Ruslan Salakhutdinov、Abdelrahman Mohamed在HuBERT：通过隐藏单元的掩蔽预测进行自我监督语音表征学习中提出的。

该模型继承自PreTrainedModel。检查超类文档以了解该库为其所有模型实现的通用方法（例如下载或保存等）。

该模型是 PyTorch torch.nn.Module子类。将其用作常规 PyTorch 模块，并参阅 PyTorch 文档以了解与一般用法和行为相关的所有事项。

forward

( input_values: typing.Optional[torch.Tensor]attention_mask: typing.Optional[torch.Tensor] = Nonemask_time_indices: typing.Optional[torch.FloatTensor] = Noneoutput_attentions: typing.Optional[bool] = Noneoutput_hidden_states: typing.Optional[bool] = Nonereturn_dict: typing.Optional[bool] = None ) → transformers.modeling_outputs.BaseModelOutput or tuple(torch.FloatTensor)

Parameters

input_values (torch.FloatTensor of shape (batch_size, sequence_length)) — Float values of input raw speech waveform. Values can be obtained by loading a .flac or .wav audio file into an array of type List[float] or a numpy.ndarray, e.g. via the soundfile library (pip install soundfile). To prepare the array into input_values, the AutoProcessor should be used for padding and conversion into a tensor of type torch.FloatTensor. See Wav2Vec2Processor.call() for details.  — 输入原始语音波形的浮点值。

attention_mask (torch.LongTensor of shape (batch_size, sequence_length), optional) — Mask to avoid performing convolution and attention on padding token indices. — 避免在填充标记索引上执行卷积和注意力的掩码。
Mask values selected in [0, 1]:
1 for tokens that are not masked,
0 for tokens that are masked.

##### 什么是注意力面具

attention_mask仅当相应的处理器具有config.return_attention_mask == True. 对于处理器具有 的所有模型config.return_attention_mask == False，例如 hubert-base，attention_mask不应传递，以避免在进行批量推理时性能下降。对于这样的模型，input_values应该简单地用 0 填充并在没有 的情况下传递attention_mask。请注意，根据是否input_values填充，这些模型也会产生略有不同的结果。

output_attentions ( bool,可选) — 是否返回所有注意力层的注意力张量。attentions有关更多详细信息，请参阅返回的张量。

output_hidden_​​states ( bool,可选) — 是否返回所有层的隐藏状态。hidden_states有关更多详细信息，请参阅返回的张量。

return_dict ( bool,可选) — 是否返回 ModelOutput而不是普通元组。

Returns

transformers.modeling_outputs.BaseModelOutput or tuple(torch.FloatTensor)



A transformers.modeling_outputs.BaseModelOutput or a tuple of torch.FloatTensor (if return_dict=False is passed or when config.return_dict=False) comprising various elements depending on the configuration (HubertConfig) and inputs.

last_hidden_state (torch.FloatTensor of shape (batch_size, sequence_length, hidden_size)) — Sequence of hidden-states at the output of the last layer of the model. — 模型最后一层输出的隐藏状态序列。

hidden_states (tuple(torch.FloatTensor), optional, returned when output_hidden_states=True is passed or when config.output_hidden_states=True) — Tuple of torch.FloatTensor (one for the output of the embeddings, if the model has an embedding layer, + one for the output of each layer) of shape (batch_size, sequence_length, hidden_size).可选output_hidden_states=True, 当被传递 或 当 时返回config.output_hidden_states=True) — shape 的元组torch.FloatTensor（一个用于嵌入的输出，如果模型具有嵌入层，+一个用于每层的输出）(batch_size, sequence_length, hidden_size)。

Hidden-states of the model at the output of each layer plus the optional initial embedding outputs.每层输出处模型的隐藏状态加上可选的初始嵌入输出。



attentions (tuple(torch.FloatTensor), optional, returned when output_attentions=True is passed or when config.output_attentions=True) — Tuple of torch.FloatTensor (one for each layer) of shape (batch_size, num_heads, sequence_length, sequence_length).

注意力权重位于注意力softmax之后，用于计算自注意力头中的加权平均值。



HubertModel的转发方法覆盖了__call__特殊方法。

<font style=background:green>尽管需要在此函数中定义前向传递的配方，但应该Module 在之后调用实例而不是此函数，因为前者负责运行预处理和后处理步骤，而后者则默默地忽略它们。</font>

#### Example

In [61]:
from transformers import AutoProcessor, HubertModel
from datasets import load_dataset
import soundfile as sf

processor = AutoProcessor.from_pretrained("facebook/hubert-large-ls960-ft")
model = HubertModel.from_pretrained("facebook/hubert-large-ls960-ft")


def map_to_array(batch):
    speech, _ = sf.read(batch["file"])
    batch["speech"] = speech
    return batch


ds = load_dataset("hf-internal-testing/librispeech_asr_dummy", "clean", split="validation")
ds = ds.map(map_to_array)

input_values = processor(ds["speech"][0], return_tensors="pt").input_values  # Batch size 1
hidden_states = model(input_values).last_hidden_state

Loading the tokenizer from the `special_tokens_map.json` and the `added_tokens.json` will be removed in `transformers 5`,  it is kept for forward compatibility, but it is recommended to update your `tokenizer_config.json` by uploading it again. You will see the new `added_tokens_decoder` attribute that will store the relevant information.


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/73 [00:00<?, ? examples/s]

It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


### HubertForCTC

Hubert Model with a language modeling head on top for Connectionist Temporal Classification (CTC). Hubert was proposed in HuBERT: Self-Supervised Speech Representation Learning by Masked Prediction of Hidden Units by Wei-Ning Hsu, Benjamin Bolte, Yao-Hung Hubert Tsai, Kushal Lakhotia, Ruslan Salakhutdinov, Abdelrahman Mohamed.
（Hubert Model）language modeling在联结主义时间分类（CTC）方面处于领先地位。Hubert 是由 Wei-Ning Hsu、Benjamin Bolte、Yao-Hung Hubert Tsai、Kushal Lakhotia、Ruslan Salakhutdinov、Abdelrahman Mohamed在HuBERT：通过隐藏单元的掩蔽预测进行自我监督语音表征学习中提出的。

This model inherits from PreTrainedModel. Check the superclass documentation for the generic methods the library implements for all its model (such as downloading or saving etc.).
该模型继承自PreTrainedModel。检查超类文档以了解该库为其所有模型实现的通用方法（例如下载或保存等）。

This model is a PyTorch torch.nn.Module sub-class. Use it as a regular PyTorch Module and refer to the PyTorch documentation for all matter related to general usage and behavior.
该模型是 PyTorch torch.nn.Module子类。将其用作常规 PyTorch 模块，并参阅 PyTorch 文档以了解与一般用法和行为相关的所有事项。

#### Example

In [62]:
from transformers import AutoProcessor, HubertForCTC
from datasets import load_dataset
import torch

dataset = load_dataset("hf-internal-testing/librispeech_asr_demo", "clean", split="validation")
dataset = dataset.sort("id")
sampling_rate = dataset.features["audio"].sampling_rate

processor = AutoProcessor.from_pretrained("facebook/hubert-large-ls960-ft")
model = HubertForCTC.from_pretrained("facebook/hubert-large-ls960-ft")

# audio file is decoded on the fly
inputs = processor(dataset[0]["audio"]["array"], sampling_rate=sampling_rate, return_tensors="pt")
with torch.no_grad():
    logits = model(**inputs).logits
predicted_ids = torch.argmax(logits, dim=-1)

# transcribe speech
transcription = processor.batch_decode(predicted_ids)
transcription[0]

inputs["labels"] = processor(text=dataset[0]["text"], return_tensors="pt").input_ids

# compute loss
loss = model(**inputs).loss
round(loss.item(), 2)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating validation split: 0 examples [00:00, ? examples/s]

22.68

### HubertForSequenceClassification

Hubert Model with a sequence classification head on top (a linear layer over the pooled output) for tasks like SUPERB Keyword Spotting.
Hubert 模型的顶部有一个序列分类头（池化输出上的线性层），用于执行 SUPERB 关键字识别等任务。

Hubert was proposed in HuBERT: Self-Supervised Speech Representation Learning by Masked Prediction of Hidden Units by Wei-Ning Hsu, Benjamin Bolte, Yao-Hung Hubert Tsai, Kushal Lakhotia, Ruslan Salakhutdinov, Abdelrahman Mohamed.
Hubert 是由 Wei-Ning Hsu、Benjamin Bolte、Yao-Hung Hubert Tsai、Kushal Lakhotia、Ruslan Salakhutdinov、Abdelrahman Mohamed在HuBERT：通过隐藏单元的掩蔽预测进行自我监督语音表征学习中提出的。

This model inherits from PreTrainedModel. Check the superclass documentation for the generic methods the library implements for all its model (such as downloading or saving etc.).
该模型继承自PreTrainedModel。检查超类文档以了解该库为其所有模型实现的通用方法（例如下载或保存等）。

This model is a PyTorch torch.nn.Module sub-class. Use it as a regular PyTorch Module and refer to the PyTorch documentation for all matter related to general usage and behavior.
该模型是 PyTorch torch.nn.Module子类。将其用作常规 PyTorch 模块，并参阅 PyTorch 文档以了解与一般用法和行为相关的所有事项。

#### Example

In [63]:
from transformers import AutoFeatureExtractor, HubertForSequenceClassification
from datasets import load_dataset
import torch

dataset = load_dataset("hf-internal-testing/librispeech_asr_demo", "clean", split="validation")
dataset = dataset.sort("id")
sampling_rate = dataset.features["audio"].sampling_rate

feature_extractor = AutoFeatureExtractor.from_pretrained("superb/hubert-base-superb-ks")
model = HubertForSequenceClassification.from_pretrained("superb/hubert-base-superb-ks")

# audio file is decoded on the fly
inputs = feature_extractor(dataset[0]["audio"]["array"], sampling_rate=sampling_rate, return_tensors="pt")

with torch.no_grad():
    logits = model(**inputs).logits

predicted_class_ids = torch.argmax(logits, dim=-1).item()
predicted_label = model.config.id2label[predicted_class_ids]
predicted_label

# compute loss - target_label is e.g. "down"
target_label = model.config.id2label[0]
inputs["labels"] = torch.tensor([model.config.label2id[target_label]])
loss = model(**inputs).loss
round(loss.item(), 2)

8.53

## Wav2Vec2文档